# 🌴 Asistente Turístico de Tenerife - RAG Demo

## 1. Configuración del Entorno
Cargamos las librerías necesarias y verificamos la API Key.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if api_key:
    print(f"✅ Key cargada: {api_key[:5]}...")
else:
    print("❌ Error de Key")

✅ Key cargada: sk-pr...


In [2]:
# 0. INSTALACIÓN DE DEPENDENCIAS
# Ejecuta esto PRIMERO. Si hay actualizaciones, Reinicia el Kernel antes de seguir.
%pip install -q -U openai-agents nest_asyncio openai langchain-openai python-dotenv
print("✅ Dependencias verificadas. Si hubo instalaciones nuevas, ve a Kernel > Restart.")

Note: you may need to restart the kernel to use updated packages.
✅ Dependencias verificadas. Si hubo instalaciones nuevas, ve a Kernel > Restart.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## 2. Validación de Conectividad con OpenAI
Probamos que la llave funcione y el modelo responda.

In [3]:
from langchain_openai import ChatOpenAI

# Inicializamos el modelo
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

# Prueba 'Holística'
try:
    response = llm.invoke("Di 'Hola Tenerife' si me escuchas.")
    print(f"🤖 Respuesta Modelo: {response.content}")
except Exception as e:
    print(f"❌ Error de Conexión: {e}")

🤖 Respuesta Modelo: Hola Tenerife. ¿En qué puedo ayudarte hoy?


## 3. Fase 2: Ingesta y Vector Store 📚
Subimos el PDF `TENERIFE.pdf` a OpenAI para crear un Vector Store gestionado.

In [4]:
import os
import sys
import openai
from openai import OpenAI
from pathlib import Path

# 1. DIAGNÓSTICO DE ENTORNO (CRÍTICO)
print(f"🐍 Python Executable: {sys.executable}")
if "llm-env" not in sys.executable:
    print("⚠️ ADVERTENCIA: No parece que estés usando el entorno virtual 'llm-env'. Verifica tu Kernel.")

print(f"📦 OpenAI Version: {openai.__version__}")

client = OpenAI()

pdf_path = Path("../data/raw/TENERIFE.pdf")

if not pdf_path.exists():
    print(f"❌ No encontrado: {pdf_path}")
else:
    print(f"✅ Archivo encontrado: {pdf_path}")

    # 2. ADAPTADOR UNIVERSAL DE VECTOR STORES
    try:
        vs_manager = None
        
        # Intento 1: Ruta estándar V2 Beta
        if hasattr(client, 'beta') and hasattr(client.beta, 'vector_stores'):
            vs_manager = client.beta.vector_stores
            print("✅ Conectado vía client.beta.vector_stores")
            
        # Intento 2: Ruta raíz (posible cambio futuro o versión diferente)
        elif hasattr(client, 'vector_stores'):
            vs_manager = client.vector_stores
            print("✅ Conectado vía client.vector_stores")

        if vs_manager:
            vector_store = vs_manager.create(name="Tenerife Guide Store")
            print(f"📦 Vector Store creado: {vector_store.id}")
            
            with open(pdf_path, "rb") as f:
                file_batch = vs_manager.file_batches.upload_and_poll(
                    vector_store_id=vector_store.id, files=[f]
                )
            print(f"📄 Estado de carga: {file_batch.status}")
            print(f"🔢 Archivos procesados: {file_batch.file_counts}")
        else:
            # FALLO TOTAL: Imprimir introspección para depurar
            print("❌ ERROR: No se encuentra 'vector_stores' en el cliente OpenAI.")
            print("🔍 Depuración de atributos disponibles:")
            print(f"   client dir: {[x for x in dir(client) if 'vector' in x or 'beta' in x]}")
            if hasattr(client, 'beta'):
                print(f"   client.beta dir: {dir(client.beta)}")
            else:
                print("   client.beta NO existe.")
            raise AttributeError("No se pudo acceder a Vector Stores con esta versión de librería/entorno.")

    except Exception as e:
        print(f"❌ Excepción: {e}")

🐍 Python Executable: f:\development\Development\Master IA\LLM-large-language-models-entrega-tarea-final\llm-env\Scripts\python.exe
📦 OpenAI Version: 2.15.0
✅ Archivo encontrado: ..\data\raw\TENERIFE.pdf
✅ Conectado vía client.vector_stores
📦 Vector Store creado: vs_6967758ac54c819185f9a78ae16d9740
📄 Estado de carga: completed
🔢 Archivos procesados: FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


## 4. Fase 3: Configuración del Agente RAG 🤖
Configuramos el agente con capacidad de usar el Vector Store. 
**Nota**: Incluimos la instalación de librerías en el kernel para asegurar que el entorno de ejecución tenga todo lo necesario.

In [5]:
import nest_asyncio
nest_asyncio.apply()

# Corrección: La librería se importa como 'agents' aunque se instale como 'openai-agents'
from agents import Agent, Runner, FileSearchTool

# Recuperamos el ID del Vector Store de la fase anterior
# Fallback a un ID conocido si se ha reiniciado el kernel sin re-ejecutar la Fase 2
try:
    current_vs_id = vector_store.id
except NameError:
    current_vs_id = "vs_69661c161c5881919eadf77744958c70" # ID de ejemplo/previo
    print(f"⚠️ Usando ID recuperado/hardcodeado: {current_vs_id}")

# 1. Definir la herramienta de búsqueda de archivos
file_search_tool = FileSearchTool(vector_store_ids=[current_vs_id])

# 2. Instrucciones del Agente (System Prompt)
INSTRUCTIONS = """
Eres un experto guía turístico de Tenerife.
Usa la herramienta file_search para encontrar información en la guía PDF adjunta.
Si la información no está en el PDF, dilo claramente. No inventes respuestas.
Responde siempre en español, con un tono amable y entusiasta.
"""

# 3. Crear el Agente
agent = Agent(
    name="TenerifeGuide",
    model="gpt-4o-mini",
    instructions=INSTRUCTIONS,
    tools=[file_search_tool]
)

print(f"🤖 Agente '{agent.name}' configurado satisfactoriamente.")

🤖 Agente 'TenerifeGuide' configurado satisfactoriamente.


### Prueba Rápida del Agente
Lanzamos una pregunta de prueba para verificar que el RAG funciona.

In [6]:
async def demo_agente():
    pregunta = "¿Qué actividades puedo hacer en el Teide?"
    print(f"👤 Usuario: {pregunta}")
    
    # Ejecutamos el agente
    resultado = await Runner.run(agent, pregunta)
    print(f"🤖 Agente: {resultado.final_output}")

await demo_agente()

👤 Usuario: ¿Qué actividades puedo hacer en el Teide?
🤖 Agente: ¡Qué emoción que estés interesado en las actividades del Teide! Aquí tienes algunas recomendaciones:

1. **Subida al Pico del Teide**: Puedes ascender utilizando los teleféricos que te llevarán cerca de la cima. Desde ahí, ¡las vistas son espectaculares!

2. **Visitas Nocturnas**: Si tienes la oportunidad, considera subir por la noche. El Teide ofrece uno de los cielos estrellados más impresionantes del mundo, especialmente cuando está despejado.

3. **Centro de Visitantes**: No olvides visitar el Centro de Visitantes de El Portillo, que es gratuito y ofrece información valiosa sobre el parque.

4. **Senderismo**: Hay diversas rutas de senderismo para todos los niveles. Disfrutar del paisaje volcánico y la flora endémica es una experiencia única.

5. **Miradores**: Aprovecha para visitar los miradores como el Mirador de La Tarta, donde puedes disfrutar de vistas increíbles durante tu subida.

¡Así que prepara tu cámara y di

# 4. FASE 4: GENERACIÓN DE DATASET DE EVALUACIÓN

Para medir la calidad de nuestro RAG, necesitamos un "Gold Standard" o dataset de prueba.
En lugar de escribirlo a mano, usaremos un **Agente Auxiliar ('DatasetBuilder')**.

Este agente leerá el PDF y generará pares de Pregunta/Respuesta con referencias exactas.
Esto asegura que las pruebas estén alineadas con el contenido real del documento.

In [7]:
from typing import List
from pydantic import BaseModel
import json

# Modelo de datos para cada item de evaluación
class EvalItem(BaseModel):
    question: str
    answer: str
    references: List[str]

NUM_EXAMPLES = 5  # Número de pares a generar (puedes aumentar esto luego)
MODEL_GENERATOR = "gpt-4o-mini" # Usamos el mismo modelo capaz

# Definición del Agente Generador
dataset_agent = Agent(
    name="DatasetBuilder",
    model=MODEL_GENERATOR,
    instructions=(
        "Eres un generador de datasets de QA (Pregunta/Respuesta) basado en un PDF accesible vía file_search. "
        "Crea preguntas variadas (literales, síntesis, curiosidades) sobre Tenerife y respuestas concisas apoyadas en el texto. "
        "Debes devolver un JSON con una lista de objetos que sigan la estructura {question, answer, references}. "
        "Las 'references' deben ser citas textuales exactas del PDF que soporten la respuesta. NO inventes texto."
    ),
    tools=[file_search_tool], # Reutilizamos la tool configurada en Fase 3
    output_type=List[EvalItem],
)

print("✅ Agente DatasetBuilder configurado.")

✅ Agente DatasetBuilder configurado.


In [8]:
# Prompt para disparar la generación
dataset_prompt = (
    f"Genera {NUM_EXAMPLES} pares de pregunta/respuesta basados únicamente en el PDF de Tenerife. "
    "Incluye las referencias textuales relevantes en el campo 'references'. "
    "Responde solo con un JSON válido."
)

print(f"⏳ Generando {NUM_EXAMPLES} ejemplos de evaluación... (Esto puede tardar unos segundos)")

try:
    # Ejecución del agente
    dataset_result = await Runner.run(dataset_agent, dataset_prompt)
    
    # Extracción del resultado tipado
    eval_dataset = dataset_result.final_output_as(List[EvalItem])
    
    print(f"\n✅ Generados {len(eval_dataset)} ejemplos:")
    for i, item in enumerate(eval_dataset, 1):
        print(f"[{i}] P: {item.question}")
        print(f"    R: {item.answer[:100]}...")

except Exception as e:
    print(f"❌ Error generando dataset: {e}")


⏳ Generando 5 ejemplos de evaluación... (Esto puede tardar unos segundos)

✅ Generados 5 ejemplos:
[1] P: ¿Cuál es la capital de Tenerife?
    R: La capital de la isla es Santa Cruz de Tenerife....
[2] P: ¿Cuáles son las características del Teide?
    R: El Teide es el pico más alto de España con 3.718 m, ubicado en un parque nacional considerado de obl...
[3] P: ¿Qué tipo de clima tiene Tenerife?
    R: Tenerife tiene un clima diverso, con zonas de playa de arena blanca en el sur y más de arena negra e...
[4] P: ¿Cuáles son las playas más famosas de Tenerife?
    R: Entre las playas más famosas están Playa de Las Teresitas, Playa del Camisón y Playa de Los Cristian...
[5] P: ¿Qué actividades se pueden realizar en Tenerife?
    R: Se pueden realizar diversas actividades como senderismo en el Parque Rural de Anaga, visitar el Loro...


In [9]:
# Guardar el dataset en disco para no regenerarlo siempre
DATASET_PATH = Path("eval_dataset_tenerife.json")

if 'eval_dataset' in locals() and eval_dataset:
    with open(DATASET_PATH, "w", encoding="utf-8") as f:
        json.dump([item.model_dump() for item in eval_dataset], f, ensure_ascii=False, indent=2)
    print(f"💾 Dataset guardado en: {DATASET_PATH.absolute()}")
else:
    print("⚠️ No hay dataset para guardar.")

💾 Dataset guardado en: f:\development\Development\Master IA\LLM-large-language-models-entrega-tarea-final\notebooks\eval_dataset_tenerife.json


# 5. FASE 5: VERIFICACIÓN Y MÉTRICAS (JUEZ LLM)

Ya tenemos Agente y Dataset. Ahora necesitamos saber **qué tan bien** funciona.
Usaremos un segundo LLM (Juez) para calificar cada respuesta del Agente según:
1.  **Faithfulness (Fidelidad):** ¿Se inventó algo?
2.  **Answer Correctness:** ¿Coincide con la respuesta esperada?
3.  **Context Precision:** ¿Los trozos de PDF recuperados eran útiles?

In [10]:
import asyncio
import statistics
from typing import Dict, Any, List
from pydantic import BaseModel

# Aseguramos imports
from agents import Agent, Runner
import nest_asyncio
nest_asyncio.apply()

# Definición de la Salida del Juez
class JudgeScores(BaseModel):
    faithfulness: float
    answer_correctness: float
    context_precision: float
    reasoning: str

# Definición del Juez (usamos gpt-4o-mini por ser rápido y capaz)
judge_agent = Agent(
    name="EvalJudge",
    model="gpt-4o-mini",
    instructions=(
        "Actúas como un juez evaluador de sistemas RAG. "
        "Debes generar puntuaciones (0.0 a 1.0) para: "
        "1. Faithfulness: ¿La respuesta se basa solo en el contexto recuperado? (1=Sí, 0=Alucinación total). "
        "2. Answer Correctness: ¿La respuesta del agente coincide semánticamente con la respuesta de referencia? "
        "3. Context Precision: ¿Los contextos recuperados son relevantes para la pregunta? "
        "Devuelve un JSON con los campos numéricos y un breve 'reasoning'."
    ),
    output_type=JudgeScores
)

print("✅ Agente Juez configurado.")

✅ Agente Juez configurado.


In [11]:
# Función auxiliar para extraer el texto de los contextos recuperados
def extract_contexts(run_result) -> List[str]:
    contexts = []
    # Inspeccionamos items generados buscando llamadas a tools
    if hasattr(run_result, 'new_items'):
        for item in run_result.new_items:
            raw = getattr(item, "raw_item", None)
            # Detectar llamada a file_search
            if getattr(raw, "type", None) == "file_search_call":
                for res in getattr(raw, "results", []) or []:
                    text = getattr(res, "text", None) or getattr(res, "content", None)
                    if text: contexts.append(text)
    return contexts

# Función que evalúa UNA sola pregunta
async def evaluate_example(agent_to_test: Agent, example: EvalItem):
    # 1. El Agente a testear responde la pregunta
    run_res = await Runner.run(agent_to_test, example.question)
    agent_answer = run_res.final_output
    contexts = extract_contexts(run_res)
    
    # 2. El Juez evalúa la calidad
    context_text = "\n---\n".join(contexts[:3]) # Top 3 contextos
    judge_prompt = (
        f"Pregunta: {example.question}\n"
        f"Respuesta Agente: {agent_answer}\n"
        f"Respuesta Referencia: {example.answer}\n"
        f"Contextos Recuperados: {context_text}\n"
        "Evalúa con rigor."
    )
    
    judge_res = await Runner.run(judge_agent, judge_prompt)
    scores = judge_res.final_output_as(JudgeScores)
    
    return {
        "question": example.question,
        "agent_answer": agent_answer,
        "reference": example.answer,
        "faithfulness": scores.faithfulness,
        "correctness": scores.answer_correctness,
        "ctx_precision": scores.context_precision,
        "reasoning": scores.reasoning
    }

In [12]:
# Ejecución Masiva de la Evaluación
print(f"⏳ Evaluando los {len(eval_dataset)} ejemplos con el Juez... Estiman 10-20 segundos.")

results = []
for ex in eval_dataset:
    res = await evaluate_example(agent, ex)
    results.append(res)
    # Pequeño print de progreso
    print(f". Completado: {ex.question[:30]}... -> F:{res['faithfulness']} C:{res['correctness']}")

print("\n✅ Evaluación Finalizada.")

⏳ Evaluando los 5 ejemplos con el Juez... Estiman 10-20 segundos.
. Completado: ¿Cuál es la capital de Tenerif... -> F:1.0 C:1.0
. Completado: ¿Cuáles son las característica... -> F:1.0 C:0.8
. Completado: ¿Qué tipo de clima tiene Tener... -> F:0.9 C:0.7
. Completado: ¿Cuáles son las playas más fam... -> F:0.8 C:0.7
. Completado: ¿Qué actividades se pueden rea... -> F:0.9 C:0.8

✅ Evaluación Finalizada.


In [13]:
# Reporte de Resultados
try:
    import pandas as pd
    df_results = pd.DataFrame(results)
    print("\nPromedios Generales:")
    print(df_results[["faithfulness", "correctness", "ctx_precision"]].mean())
    display(df_results) # Pretty print en notebook
except ImportError:
    # Fallback si no hay pandas
    print("\nResultados Detallados:")
    for r in results:
        print(r)


Resultados Detallados:
{'question': '¿Cuál es la capital de Tenerife?', 'agent_answer': 'La capital de Tenerife es Santa Cruz de Tenerife. ¡Es un lugar lleno de vida y con muchos lugares interesantes para explorar! Si tienes algún otro dato que te gustaría saber sobre Santa Cruz o Tenerife en general, ¡pregúntame!', 'reference': 'La capital de la isla es Santa Cruz de Tenerife.', 'faithfulness': 1.0, 'correctness': 1.0, 'ctx_precision': 1.0, 'reasoning': 'La respuesta del agente se basa únicamente en el contexto recuperado, que menciona que la capital de Tenerife es Santa Cruz de Tenerife. Además, la respuesta coincide semánticamente con la respuesta de referencia. Los contextos recuperados son pertinentes, ya que la pregunta se refiere a la capital de Tenerife.'}
{'question': '¿Cuáles son las características del Teide?', 'agent_answer': 'El Teide es el pico más alto de España, alcanzando los 3,718 metros sobre el nivel del mar. Se encuentra situado en el Parque Nacional del Teide y e

# 6. FASE 6: FUNCTION CALLING (CLIMA)

Implementamos una herramienta externa (`get_weather`) usando el decorador `@function_tool`.
Esto permite al Agente invocar código Python real (o simulado) para obtener estructuras JSON.

In [14]:
import random
import datetime
import json
from agents import function_tool  # Importante: Decorador para herramientas

# 1. Definimos la función decorada
@function_tool
def get_weather(location: str, date: str = None) -> str:
    """
    Obtiene la previsión del tiempo para una ubicación y fecha dadas.
    
    Args:
        location: Ciudad o lugar (ej: 'Tenerife').
        date: Fecha en formato YYYY-MM-DD o 'hoy'.
    """
    # Log solicitado por requisitos
    print(f"[TOOL LOG] Llamada a get_weather(location='{location}', date='{date}')")
    
    # Mock (Simulación)
    if not date:
        date = datetime.date.today().isoformat()
    
    climates = ["Soleado", "Parcialmente nublado", "Viento moderado"]
    temp = random.randint(20, 28)
    
    return json.dumps({
        "location": location,
        "date": date,
        "condition": random.choice(climates),
        "temperature": f"{temp}ºC",
        "note": "Simulación académica"
    })

# 2. Creamos el Agente con la herramienta correctamente configurada
weather_agent = Agent(
    name="TenerifeGuidePro",
    model="gpt-4o-mini",
    instructions=(
        "Eres el experto turístico definitivo de Tenerife. "
        "Usa file_search para responder dudas sobre lugares basándote en el PDF. "
        "Usa get_weather si el usuario pregunta por el tiempo o clima actual. "
        "Combina ambas fuentes para dar consejos complejos."
    ),
    tools=[file_search_tool, get_weather]
)

print("✅ Agente 'TenerifeGuidePro' configurado correctamente.")

✅ Agente 'TenerifeGuidePro' configurado correctamente.


In [15]:
# Prueba de Function Calling + RAG
pregunta_clima = "¿Qué tiempo hace hoy en el Teide y qué me recomiendas hacer allí según el PDF?"

print(f"👤 Usuario: {pregunta_clima}")
print("⏳ Pensando...")

try:
    resultado_completo = await Runner.run(weather_agent, pregunta_clima)
    print(f"🤖 Agente: {resultado_completo.final_output}")
except Exception as e:
    print(f"❌ Error: {e}")

👤 Usuario: ¿Qué tiempo hace hoy en el Teide y qué me recomiendas hacer allí según el PDF?
⏳ Pensando...
[TOOL LOG] Llamada a get_weather(location='Teide', date='hoy')
🤖 Agente: Hoy, en el Teide, el clima es **parcialmente nublado** con una temperatura de **24°C**. Es un buen día para explorar la belleza natural del parque.

### Recomendaciones:
1. **Subida al Teide**:
   - Puedes utilizar el **teleférico** para llegar a la cima y disfrutar de vistas espectaculares. Si prefieres una aventura más intensa, considera caminar hasta la cumbre. 
   - Si te interesa la astronomía, una subida nocturna puede ofrecer uno de los cielos estrellados más impresionantes del mundo.

2. **Miradores**:
   - Para vistas panorámicas increíbles, visita el **Mirador de La Tarta** y el **Mirador de Chipeque**, donde podrás disfrutar del "mar de nubes" si tienes suerte.

3. **Centro de Visitantes de El Portillo**:
   - Es gratuito y te proporcionará información útil sobre el parque y sus senderos. 

Disfruta d